In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="7"


In [3]:

import numpy as np
from matplotlib import pyplot as plt
import nibabel as nib
from PIL import Image
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
import torchsummary
from torchvision import transforms
import time
from sklearn.metrics import classification_report

In [4]:
#이미지 불러오기
data_path='/disk1/data_liverbound_noclip/'
name_list=os.listdir(data_path)
volume_data=[files[:-4] for files in name_list if files.startswith('volume')]
volume_data=list(set(volume_data))
volume_data.sort()
#print(volume_data)

save_png_path='/home/sumins/workspace/example/save_png/'
save_volume_path='/home/sumins/workspace/example/save_png/vol_png/'

In [5]:
volume_list=[]
for file in volume_data:
    fname=os.path.basename(file)
    print(fname)
    img_array=np.load(data_path+fname+'.npy')
    
    #print(f'img_num: {img_array.shape}')
    
    if len(img_array.shape) == 3:
        nx, ny, nz = img_array.shape
        total_slices = img_array.shape[2]
        # iterate through slices
        for current_slice in range(0, total_slices):
            volume_list.append(img_array[:,:,current_slice]) 

volume-0
volume-1
volume-10
volume-100
volume-101
volume-102
volume-103
volume-104
volume-105
volume-106
volume-107
volume-108
volume-109
volume-11
volume-110
volume-111
volume-112
volume-113
volume-114
volume-115
volume-116
volume-117
volume-118
volume-119
volume-12
volume-120
volume-121
volume-122
volume-123
volume-124
volume-125
volume-126
volume-127
volume-128
volume-129
volume-13
volume-14
volume-15
volume-16
volume-17
volume-18
volume-19
volume-2
volume-20
volume-21
volume-22
volume-23
volume-24
volume-25
volume-26
volume-27
volume-28
volume-29
volume-3
volume-30
volume-31
volume-32
volume-33
volume-34
volume-35
volume-36
volume-37
volume-38
volume-39
volume-4
volume-40
volume-41
volume-42
volume-43
volume-44
volume-45
volume-46
volume-47
volume-48
volume-49
volume-5
volume-50
volume-51
volume-52
volume-53
volume-54
volume-55
volume-56
volume-57
volume-58
volume-59
volume-6
volume-60
volume-61
volume-62
volume-63
volume-64
volume-65
volume-66
volume-67
volume-68
volume-69
volume-

In [6]:
all_labels =np.loadtxt('/home/sumins/workspace/liver_classification/all_labels.txt',dtype=int)
all_labels=all_labels.tolist()

In [7]:
WINDOW_MAX = 200
WINDOW_MIN = 0
GLOBAL_PIXEL_MEAN = 0.1

class CustomDataset(Dataset): 
  def __init__(self,volume_list,all_labels,transforms=None):
    self.volume_list=volume_list
    self.all_labels=all_labels
    self.length=len(all_labels)
    self.transforms=transforms


  def __len__(self):
    return self.length


  def __getitem__(self, idx):
    npy=self.volume_list[idx]

    npy[npy > WINDOW_MAX] = WINDOW_MAX
    npy[npy < WINDOW_MIN] = WINDOW_MIN
    
    npy = (npy - WINDOW_MIN) / (WINDOW_MAX - WINDOW_MIN)
    npy -= GLOBAL_PIXEL_MEAN
    
    if len(npy.shape)==2:
      npy=npy[:,:,np.newaxis].astype(dtype='float32')
    
    if self.transforms is not None:
      npy=self.transforms(npy)
    
    return{'npy':npy,'label':self.all_labels[idx]}

In [8]:
hyper_param_epoch=100
hyper_param_batch=8
hyper_param_learning_rate=0.0001
from torch import nn
class CNN(nn.Module):
    def __init__(self):
    	# super함수는 CNN class의 부모 class인 nn.Module을 초기화
        super(CNN, self).__init__()
        
        # batch_size = 1
        self.layer = nn.Sequential(
            # [1,1,512,512] -> [1,16,508,508]
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5),
            nn.ReLU(),
            
            # [1,16,508,508] -> [1,32,504,504]
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5),
            nn.ReLU(),
            
            # [1,32,504,504] -> [1,32,10,252,252]
            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # [1,32,252,252] -> [1,64,248,248]
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),
            nn.ReLU(),
            
            # [1,64,248,248] -> [1,64,124,124]
            nn.MaxPool2d(kernel_size=2,stride=2), 

            # [1,64,124,124] -> [1,128,120,120]
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5),
            nn.ReLU(),
            # [1,128,120,120] -> [1,128,60,60]
            nn.MaxPool2d(kernel_size=2,stride=2),    
            # [1,128,60,60] -> [1,256,56,56]
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5),
            nn.ReLU(),
            # [1,256,56,56] -> [1,256,28,28]
            nn.MaxPool2d(kernel_size=2,stride=2)         
        )
        self.fc_layer = nn.Sequential(
        	# [256*28*28]-->[100]
            nn.Linear(256*28*28,100),                                              
            nn.ReLU(),

            # [100,100] -> [100,10]
            nn.Linear(100,2)                                                   
        )       
        
    def forward(self,x):
    	# self.layer에 정의한 연산 수행
        
        out = self.layer(x)
        #print(out.shape)
        # view 함수를 이용해 텐서의 형태를 [100,나머지]로 변환
        out = out.view(hyper_param_batch,-1)
        #print(out.shape)
        # self.fc_layer 정의한 연산 수행    
        out = self.fc_layer(out)
        return out
    


In [9]:
transforms_train = transforms.Compose([transforms.ToTensor()
                                       ])
transforms_test = transforms.Compose([transforms.ToTensor()
                                       ])



In [10]:
vol_train, vol_valid, lab_train, lab_valid = train_test_split(volume_list, all_labels, test_size=0.3, shuffle=True, stratify=all_labels, random_state=34)
train_dataset=CustomDataset(volume_list=vol_train, all_labels=lab_train,transforms=transforms_train)
test_dataset=CustomDataset(volume_list=vol_valid,all_labels=lab_valid,transforms=transforms_test)
train_loader = DataLoader(train_dataset, batch_size=hyper_param_batch, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=hyper_param_batch, shuffle=True)
len(train_dataset)
len(test_dataset)

6663

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device,"/" ,torch.cuda.device_count())

custom_model=CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)

cuda:0 / 1


In [ ]:
model=CNN().to(device)
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)

checkpoint = torch.load('/home/sumins/workspace/model_check/model_3.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss_value = checkpoint['loss']
print(loss_value)

start=time.time()
model.train()
for e in range(hyper_param_epoch):
    for i_batch, item in enumerate(train_loader):
        npys = item['npy'].to(device)
        labels = item['label'].to(device)
        #print(npys)
        # Forward pass
        outputs = model(npys)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #if(i_batch+1)%10==0:
            
    print('Epoch [{}/{}],i_batch={} Loss: {:.4f}'
                .format(e + 1, hyper_param_epoch, i_batch+1, loss.item()))
    print("Time: {}sec".format(time.time()-start))
    start=time.time()
    if loss_value>loss.item():
            loss_value=loss.item()
            torch.save({
            'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, '/home/sumins/workspace/model_check/model_4.pth')

In [12]:
true_label=[]
pred_label=[]
model=CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_param_learning_rate)

checkpoint = torch.load('/home/sumins/workspace/model_check/model_4.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(loss)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for item in test_loader:
        npys = item['npy'].to(device)
        labels = item['label'].to(device)
        if(len(labels)!=8):
            break
        outputs = model(npys)
        _, predicted = torch.max(outputs.data, 1)
        true_label.extend(labels)
        pred_label.extend(predicted)
        total += len(labels)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))



tensor(0.0001, device='cuda:0', requires_grad=True)
Test Accuracy of the model on the 6656 test images: 97.47596153846153 %


In [13]:
true_labels=torch.tensor(true_label)
true_labels=true_labels.tolist()
pred_labels=torch.tensor(pred_label)
pred_labels=pred_labels.tolist()
print(classification_report(true_labels,pred_labels))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4560
           1       0.97      0.95      0.96      2096

    accuracy                           0.97      6656
   macro avg       0.97      0.97      0.97      6656
weighted avg       0.97      0.97      0.97      6656



In [1]:

model=CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_param_learning_rate)

checkpoint = torch.load('/home/sumins/workspace/model_check/model_3.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print(loss)
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for item in test_loader:
        npys = item['npy'].to(device)
        labels = item['label'].to(device)
        if(len(labels)!=8):
            break
        outputs = model(npys)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))



NameError: name 'CNN' is not defined